In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder, StandardScaler

# 데이터 불러오기

In [2]:
# 데이터 로드
data = pd.read_csv('~/Documents/ds_study/EPL_prediction_ML/data/merge_data.csv')
data

,Date,HomeTeam,AwayTeam,FTR,HTR,HS,AS,HST,AST,HC,AC,HF,AF,HY,AY,HR,AR,home_xG,away_xG
0,2017-08-11,Arsenal,Leicester,H,D,27,6,10,3,9,4,9,12,0,1,0,0,2.543290,1.464950
1,2017-08-12,Brighton,Man City,A,D,6,14,2,4,3,10,6,9,0,2,0,0,0.276343,1.867510
2,2017-08-12,Chelsea,Burnley,A,A,19,10,6,5,8,5,16,11,3,3,2,0,1.356510,0.564237
3,2017-08-12,Crystal Palace,Huddersfield,A,A,14,8,4,6,12,9,7,19,1,3,0,0,0.988934,1.736570
4,2017-08-12,Everton,Stoke,H,H,9,9,4,1,6,7,13,10,1,1,0,0,0.720574,0.276440
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2107,2023-05-28,Everton,Bournemouth,H,D,13,7,6,2,11,12,9,3,1,3,0,0,1.129330,0.432084
2108,2023-05-28,Leeds,Tottenham,A,A,19,11,2,7,7,5,12,3,3,0,0,0,1.191510,2.033150
2109,2023-05-28,Leicester,West Ham,H,H,13,16,4,3,8,10,3,5,1,1,0,0,2.030130,0.970352
2110,2023-05-28,Man United,Fulham,H,D,21,10,8,3,14,10,5,4,1,2,0,0,2.747660,1.881670


In [3]:
# 결측지 있는지 확인

missing_values = data.isnull().sum()
print(missing_values)

Date        0
HomeTeam    0
AwayTeam    0
FTR         0
HTR         0
HS          0
AS          0
HST         0
AST         0
HC          0
AC          0
HF          0
AF          0
HY          0
AY          0
HR          0
AR          0
home_xG     0
away_xG     0
dtype: int64


In [4]:
#데이터 정확히 확인
print(data.head())

print(data.describe())

         Date        HomeTeam      AwayTeam FTR HTR  HS  AS  HST  AST  HC  AC  \
0  2017-08-11         Arsenal     Leicester   H   D  27   6   10    3   9   4   
1  2017-08-12        Brighton      Man City   A   D   6  14    2    4   3  10   
2  2017-08-12         Chelsea       Burnley   A   A  19  10    6    5   8   5   
3  2017-08-12  Crystal Palace  Huddersfield   A   A  14   8    4    6  12   9   
4  2017-08-12         Everton         Stoke   H   H   9   9    4    1   6   7   

   HF  AF  HY  AY  HR  AR   home_xG   away_xG  
0   9  12   0   1   0   0  2.543290  1.464950  
1   6   9   0   2   0   0  0.276343  1.867510  
2  16  11   3   3   2   0  1.356510  0.564237  
3   7  19   1   3   0   0  0.988934  1.736570  
4  13  10   1   1   0   0  0.720574  0.276440  
                HS           AS          HST          AST           HC  \
count  2112.000000  2112.000000  2112.000000  2112.000000  2112.000000   
mean     13.580966    11.305398     4.660038     3.970644     6.310606   
std

# 데이터 전처리

In [5]:
# 범주형 데이터를 원핫 인코딩으로 변환하는 함수
def preprocess_features(data, categorical_features, numerical_features, encoder=None, scaler=None):

    
# 범주형 데이터 원핫 인코딩
    if encoder is None: # encoder가 제공되지 않았다면 새로운 OneHotEncoder 객체를 생성후 데이터에 fit_transform 적용
        encoder = OneHotEncoder()
        encoded_categorical = encoder.fit_transform(data[categorical_features])
    else: # encoder가 제공되었다면 기존 인코더를 사용하여 데이터에 transform 적용
        encoded_categorical = encoder.transform(data[categorical_features])
    encoded_categorical = pd.DataFrame(encoded_categorical.toarray(), columns=encoder.get_feature_names_out(categorical_features))

 # 숫자형 데이터 스케일링
    if scaler is None:  # scaler가 제공되지 않았다면 새로운 StandardScaler 객체를 생성후 데이터에 fit_transform 적용
        scaler = StandardScaler()
        scaled_numerical = scaler.fit_transform(data[numerical_features])
    else:   # scaler가 제공되었다면 기존 스케일러를 사용하여 데이터에 transform 적용
        scaled_numerical = scaler.transform(data[numerical_features])
    scaled_numerical = pd.DataFrame(scaled_numerical, columns=numerical_features)

# 변환된 데이터 결합
    preprocessed_data = pd.concat([encoded_categorical, scaled_numerical], axis=1)
    # 결합된 데이터프레임과 사용된 encoder 및 scaler 객체를 반환
    return preprocessed_data, encoder, scaler

In [6]:
# 범주형 및 숫자형 특성 정의
categorical_features = ['HomeTeam', 'AwayTeam']  # 범주형 특성
numerical_features = ['HS', 'AS', 'HST', 'AST', 'HF', 'AF', 'HC', 'AC', 'HY', 'AY', 'HR', 'AR', 'home_xG', 'away_xG']  # 숫자형 특성

# 독립 변수 및 종속 변수 설정

In [12]:
# 전처리된 데이터셋 생성
X, encoder, scaler = preprocess_features(data, categorical_features, numerical_features)
# 타겟 변수
y = data['FTR']

# 모델 훈련 및 평가

In [13]:
# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [14]:
# 행 수 확인
print("X_train 행 수:", len(X_train))
print("y_train 행 수:", len(y_train))

# 일치 여부 확인
if len(X_train) == len(y_train):
    print("행 수가 일치합니다.")
else:
    print("행 수가 일치하지 않습니다. 데이터 확인이 필요합니다.")

X_train 행 수: 1689
y_train 행 수: 1689
행 수가 일치합니다.


In [67]:
# 모델 및 하이퍼파라미터 튜닝
from sklearn.model_selection import GridSearchCV

svc = SVC(kernel='linear')
param_grid = {
    'C': [0.01, 0.1, 1, 10],
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    'degree': [2, 3, 4],
    'gamma': ['scale', 'auto'],
    'coef0': [0.0, 0.5, 1.0],
    'class_weight': [None, 'balanced'],
    'max_iter': [1000, 10000, -1]
}
grid_search = GridSearchCV(svc, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

# 최적의 매개변수 출력
print("Best parameters found: ", grid_search.best_params_)

C:\Users\VICTUS\miniconda3\Lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
C:\Users\VICTUS\miniconda3\Lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
C:\Users\VICTUS\miniconda3\Lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
C:\Users\VICTUS\miniconda3\Lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
C:\Users\VICTUS\miniconda3\Lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (

Best parameters found:  {'C': 10, 'class_weight': None, 'coef0': 0.0, 'degree': 2, 'gamma': 'auto', 'kernel': 'rbf', 'max_iter': 10000}


# -> 거의 작업이 10분 걸려서 복잡도를 감소시켜야 함

In [68]:
# 과적합 검사
from sklearn.model_selection import cross_val_score

# 교차 검증 수행
svc = SVC(**grid_search.best_params_)
scores = cross_val_score(svc, X_train, y_train, cv=5)  # 5겹 교차 검증
print("Cross-validation scores:", scores)
print("Average score:", scores.mean())

Cross-validation scores: [0.65202703 0.64189189 0.67905405 0.63389831 0.68135593]
Average score: 0.6576454420522218


In [69]:
# 데이터 분할: 학습 데이터 70%, 테스트 데이터 30%
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [70]:
# 특성 스케일링
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [88]:
# 모델 초기화 및 학습
svc = SVC(C=1, class_weight=None, coef0=0.0, degree=2, gamma='auto', kernel='rbf', max_iter=10000)
svc.fit(X_train_scaled, y_train)

SVC(C=1, degree=2, gamma='auto', max_iter=10000)

In [89]:
from sklearn.metrics import f1_score, confusion_matrix

# 훈련 세트와 테스트 세트에서 성능 평가
y_pred_train = svc.predict(X_train)
y_pred_test = svc.predict(X_test)
print("Training F1 score:", f1_score(y_train, y_pred_train, average='macro'))
print("Test F1 score:", f1_score(y_test, y_pred_test, average='macro'))

C:\Users\VICTUS\miniconda3\Lib\site-packages\sklearn\base.py:458: UserWarning: X has feature names, but SVC was fitted without feature names
  warnings.warn(
C:\Users\VICTUS\miniconda3\Lib\site-packages\sklearn\base.py:458: UserWarning: X has feature names, but SVC was fitted without feature names
  warnings.warn(


Training F1 score: 0.6082145610541851
Test F1 score: 0.5804596382411717


In [90]:
# F1 점수를 사용하여 5-겹 교차 검증 수행
from sklearn.pipeline import make_pipeline
pipeline = make_pipeline(StandardScaler(), SVC(C= 1, gamma= 'scale', kernel= 'linear'))

cross_val_f1_scores = cross_val_score(pipeline, X, y, cv=5, scoring='f1_macro')
print("교차 검증 F1 점수:", cross_val_f1_scores)
print("평균 F1 점수:", cross_val_f1_scores.mean())

교차 검증 F1 점수: [0.56511292 0.57167127 0.5924667  0.63082227 0.52852454]
평균 F1 점수: 0.5777195391963644


In [91]:
# 혼동 행렬 출력(성능 평가)
print("Confusion Matrix (Train):\n", confusion_matrix(y_train, y_pred_train))
print("Confusion Matrix (Test):\n", confusion_matrix(y_test, y_pred_test))

Confusion Matrix (Train):
 [[393  37  57]
 [126  81 135]
 [ 83  34 532]]
Confusion Matrix (Test):
 [[173  12  23]
 [ 57  29  61]
 [ 40  24 215]]


# 예측 수행

In [92]:
svc.predict(X_test)

C:\Users\VICTUS\miniconda3\Lib\site-packages\sklearn\base.py:458: UserWarning: X has feature names, but SVC was fitted without feature names
  warnings.warn(


array(['H', 'H', 'H', 'H', 'A', 'H', 'H', 'A', 'D', 'H', 'H', 'A', 'H',
       'H', 'A', 'D', 'H', 'A', 'H', 'A', 'A', 'A', 'H', 'H', 'A', 'A',
       'A', 'A', 'D', 'D', 'H', 'A', 'H', 'H', 'H', 'A', 'A', 'A', 'D',
       'H', 'H', 'D', 'H', 'A', 'H', 'A', 'A', 'A', 'A', 'A', 'D', 'H',
       'H', 'A', 'H', 'A', 'H', 'A', 'A', 'H', 'A', 'H', 'A', 'A', 'H',
       'H', 'A', 'H', 'H', 'H', 'A', 'H', 'H', 'H', 'H', 'A', 'A', 'A',
       'H', 'H', 'A', 'H', 'H', 'A', 'H', 'A', 'D', 'H', 'A', 'H', 'H',
       'H', 'D', 'H', 'A', 'H', 'A', 'A', 'A', 'H', 'A', 'A', 'H', 'H',
       'H', 'A', 'H', 'H', 'H', 'H', 'D', 'A', 'A', 'D', 'H', 'H', 'H',
       'D', 'D', 'H', 'A', 'H', 'H', 'H', 'D', 'A', 'A', 'D', 'A', 'D',
       'A', 'H', 'H', 'A', 'A', 'H', 'A', 'A', 'H', 'A', 'D', 'A', 'D',
       'A', 'A', 'H', 'H', 'D', 'A', 'D', 'H', 'A', 'A', 'A', 'H', 'H',
       'H', 'H', 'A', 'H', 'A', 'D', 'A', 'A', 'H', 'H', 'H', 'A', 'A',
       'H', 'A', 'A', 'A', 'H', 'A', 'A', 'H', 'A', 'A', 'H', 'H

In [93]:
# 모델 초기화 시 probability 매개변수를 True로 설정(기본적으로 False로 되어있었으나 True로 설정했을때 작업시간이 오래걸려 위애서는 False로 설정함)
svc = SVC(kernel='linear', probability=True)
svc.fit(X_train, y_train)

Y_predict = svc.predict_proba(X_test)
Y_predict = pd.DataFrame(Y_predict, columns=['Away Team', 'Draw', 'Home Team'])

display((Y_predict * 100).head(10))

,Away Team,Draw,Home Team
0,16.114974,21.479227,62.405799
1,16.860313,27.327521,55.812166
2,17.877234,43.314390,38.808377
3,16.927481,22.131539,60.940980
4,57.961203,32.332741,9.706056
5,2.329280,17.454624,80.216096
6,0.142128,4.762665,95.095207
7,51.076828,30.552665,18.370507
8,34.329274,47.312615,18.358111
9,27.258629,27.505073,45.236298


In [94]:
# 기본적인 팀 정보만을 유지, 나머지 통계는 초기화하여 모델이 이를 예측하게끔 준비하는 과정

fixtures = pd.read_csv('~/Documents/ds_study/EPL_prediction_ML/data/merge_data.csv')
fixtures = fixtures.drop(['Date'], axis=1)
fixtures.columns.values[0:2] = ['HomeTeam', 'AwayTeam']
# 나머지 열 초기화
fixtures['HS'] = 0
fixtures['AS'] = 0
fixtures['HST'] = 0
fixtures['AST'] = 0
fixtures['HF'] = 0
fixtures['AF'] = 0
fixtures['HC'] = 0
fixtures['AC'] = 0
fixtures['HY'] = 0
fixtures['AY'] = 0
fixtures['HR'] = 0
fixtures['AR'] = 0
fixtures['home_xG'] = 0
fixtures['away_xG'] = 0

In [95]:
display(fixtures.head(10))

,HomeTeam,AwayTeam,FTR,HTR,HS,AS,HST,AST,HC,AC,HF,AF,HY,AY,HR,AR,home_xG,away_xG
0,Arsenal,Leicester,H,D,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Brighton,Man City,A,D,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Chelsea,Burnley,A,A,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Crystal Palace,Huddersfield,A,A,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Everton,Stoke,H,H,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,Southampton,Swansea,D,D,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,Watford,Liverpool,D,H,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,West Brom,Bournemouth,H,H,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,Man United,West Ham,H,H,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,Newcastle,Tottenham,A,D,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [96]:
# 똑같이 위에서 똑같이 수행한 원핫 인코딩 실행
preprocessedFixtures, _, _ = preprocess_features(data, categorical_features, numerical_features, encoder=None, scaler=None)

In [97]:
# 다시 한번 학습데이터와 예측데이터의 컬럼이 일치하는지 확인 

# 학습 데이터셋 컬럼 확인
train_columns = X_train.columns
print("학습 데이터셋 컬럼:")
print(train_columns)

# 예측 데이터셋 컬럼 확인
predict_columns = preprocessedFixtures.columns
print("예측 데이터셋 컬럼:")
print(predict_columns)

학습 데이터셋 컬럼:
Index(['HomeTeam_Arsenal', 'HomeTeam_Aston Villa', 'HomeTeam_Bournemouth',
       'HomeTeam_Brentford', 'HomeTeam_Brighton', 'HomeTeam_Burnley',
       'HomeTeam_Cardiff', 'HomeTeam_Chelsea', 'HomeTeam_Crystal Palace',
       'HomeTeam_Everton', 'HomeTeam_Fulham', 'HomeTeam_Huddersfield',
       'HomeTeam_Leeds', 'HomeTeam_Leicester', 'HomeTeam_Liverpool',
       'HomeTeam_Man City', 'HomeTeam_Man United', 'HomeTeam_Newcastle',
       'HomeTeam_Norwich', 'HomeTeam_Nott'm Forest',
       'HomeTeam_Sheffield United', 'HomeTeam_Southampton', 'HomeTeam_Stoke',
       'HomeTeam_Swansea', 'HomeTeam_Tottenham', 'HomeTeam_Watford',
       'HomeTeam_West Brom', 'HomeTeam_West Ham', 'HomeTeam_Wolves',
       'AwayTeam_Arsenal', 'AwayTeam_Aston Villa', 'AwayTeam_Bournemouth',
       'AwayTeam_Brentford', 'AwayTeam_Brighton', 'AwayTeam_Burnley',
       'AwayTeam_Cardiff', 'AwayTeam_Chelsea', 'AwayTeam_Crystal Palace',
       'AwayTeam_Everton', 'AwayTeam_Fulham', 'AwayTeam_Huddersfield

In [98]:
svc.predict(preprocessedFixtures)

array(['H', 'A', 'A', ..., 'H', 'H', 'A'], dtype=object)

In [99]:
Result = fixtures.drop(['HS', 'AS', 'HST', 'HF', 'AF', 'HY', 
                        'AY', 'HR', 'AR', 'HC', 'AC', 'AST', 'HC', 'AC','home_xG', 'away_xG'], axis=1)
Result.head(10)

,HomeTeam,AwayTeam,FTR,HTR
0,Arsenal,Leicester,H,D
1,Brighton,Man City,A,D
2,Chelsea,Burnley,A,A
3,Crystal Palace,Huddersfield,A,A
4,Everton,Stoke,H,H
5,Southampton,Swansea,D,D
6,Watford,Liverpool,D,H
7,West Brom,Bournemouth,H,H
8,Man United,West Ham,H,H
9,Newcastle,Tottenham,A,D


In [100]:
fixturePredictedProbability = svc.predict_proba(preprocessedFixtures) *100
fixturePredictedProbability = pd.DataFrame(fixturePredictedProbability, columns=['Away win %','Draw %','Home win %'])

display(fixturePredictedProbability)

,Away win %,Draw %,Home win %
0,19.095527,23.216733,57.687740
1,61.719421,26.615337,11.665242
2,53.289771,37.096171,9.614057
3,65.472538,28.242035,6.285427
4,5.598696,21.013578,73.387727
...,...,...,...
2107,10.749909,13.306051,75.944040
2108,73.106507,24.470182,2.423311
2109,3.106297,16.749056,80.144646
2110,5.152717,22.390524,72.456759


In [101]:
final = pd.concat([Result, fixturePredictedProbability], axis = 1)
final.head(20)

,HomeTeam,AwayTeam,FTR,HTR,Away win %,Draw %,Home win %
0,Arsenal,Leicester,H,D,19.095527,23.216733,57.687740
1,Brighton,Man City,A,D,61.719421,26.615337,11.665242
2,Chelsea,Burnley,A,A,53.289771,37.096171,9.614057
3,Crystal Palace,Huddersfield,A,A,65.472538,28.242035,6.285427
4,Everton,Stoke,H,H,5.598696,21.013578,73.387727
5,Southampton,Swansea,D,D,52.588631,37.286198,10.125171
6,Watford,Liverpool,D,H,58.658276,27.662692,13.679032
7,West Brom,Bournemouth,H,H,22.567494,33.417929,44.014577
8,Man United,West Ham,H,H,2.752803,27.576179,69.671018
9,Newcastle,Tottenham,A,D,78.697370,15.832655,5.469974


In [102]:
df = pd.read_csv('~/Documents/ds_study/EPL_prediction_ML/data/merge_data.csv')

# Date 컬럼만 출력
selected_columns = ['Date'] 
selected_df = df[selected_columns]

print(selected_df)

            Date
0     2017-08-11
1     2017-08-12
2     2017-08-12
3     2017-08-12
4     2017-08-12
...          ...
2107  2023-05-28
2108  2023-05-28
2109  2023-05-28
2110  2023-05-28
2111  2023-05-28

[2112 rows x 1 columns]


In [103]:
selected_df.reset_index(drop=True, inplace=True)
final.reset_index(drop=True, inplace=True)
# selected_df를 final의 앞에 붙이기
df = pd.concat([selected_df, final], axis=1)
df.head(20)

,Date,HomeTeam,AwayTeam,FTR,HTR,Away win %,Draw %,Home win %
0,2017-08-11,Arsenal,Leicester,H,D,19.095527,23.216733,57.687740
1,2017-08-12,Brighton,Man City,A,D,61.719421,26.615337,11.665242
2,2017-08-12,Chelsea,Burnley,A,A,53.289771,37.096171,9.614057
3,2017-08-12,Crystal Palace,Huddersfield,A,A,65.472538,28.242035,6.285427
4,2017-08-12,Everton,Stoke,H,H,5.598696,21.013578,73.387727
5,2017-08-12,Southampton,Swansea,D,D,52.588631,37.286198,10.125171
6,2017-08-12,Watford,Liverpool,D,H,58.658276,27.662692,13.679032
7,2017-08-12,West Brom,Bournemouth,H,H,22.567494,33.417929,44.014577
8,2017-08-13,Man United,West Ham,H,H,2.752803,27.576179,69.671018
9,2017-08-13,Newcastle,Tottenham,A,D,78.697370,15.832655,5.469974


In [104]:
# 컬럼 이름 수정, 추가, 순서 변경

df.drop('HTR', axis=1, inplace=True)
df.rename(columns={'FTR': 'Real result'}, inplace=True)
df['Predicted result'] = df[['Away win %', 'Draw %', 'Home win %']].idxmax(axis=1).map({
    'Away win %': 'A',
    'Draw %': 'D',
    'Home win %': 'H'
})

new = ['Date', 'HomeTeam', 'AwayTeam', 'Away win %', 'Draw %', 'Home win %', 'Real result', 'Predicted result']
df = df[new]

df

,Date,HomeTeam,AwayTeam,Away win %,Draw %,Home win %,Real result,Predicted result
0,2017-08-11,Arsenal,Leicester,19.095527,23.216733,57.687740,H,H
1,2017-08-12,Brighton,Man City,61.719421,26.615337,11.665242,A,A
2,2017-08-12,Chelsea,Burnley,53.289771,37.096171,9.614057,A,A
3,2017-08-12,Crystal Palace,Huddersfield,65.472538,28.242035,6.285427,A,A
4,2017-08-12,Everton,Stoke,5.598696,21.013578,73.387727,H,H
...,...,...,...,...,...,...,...,...
2107,2023-05-28,Everton,Bournemouth,10.749909,13.306051,75.944040,H,H
2108,2023-05-28,Leeds,Tottenham,73.106507,24.470182,2.423311,A,A
2109,2023-05-28,Leicester,West Ham,3.106297,16.749056,80.144646,H,H
2110,2023-05-28,Man United,Fulham,5.152717,22.390524,72.456759,H,H
